In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json
import requests
from pymongo import MongoClient
import numpy as np
from collections import defaultdict
import pickle

In [9]:
from webscrape_util.scrape_util import (load_json,
                                        write_json,
                                        selenium_instagram_login,
                                        setup_mongo_client)


In [10]:
client, collection = setup_mongo_client('instascrape', 'likes')

# Making dictionary of top posts for top influencers

In [70]:
hashtags_likes_dict = load_json('data/hashtags_likes_dict.json')
with open('data/top10fromallcentralities.pkl') as myfile:
    top = pickle.load(myfile)

In [4]:
# Collect posts for all top influencers
influencers_top = {}
for k,v in hashtags_likes_dict.iteritems():
    if k in top:
        influencers_top[k] = v

In [5]:
# Collect most liked post for all top influencers
influencers_top_post = {}
for k,v in influencers_top.iteritems():
    influencers_top_post[k] = {'max_likes':0, 'post': None}
    for i in range(len(v['likes'])):
        if v['likes'][i] > influencers_top_post[k]['max_likes']:
            influencers_top_post[k]['max_likes'] = v['likes'][i]
            influencers_top_post[k]['post'] = v['posts'][i]

In [13]:
top.remove("SelenaGomez") # Remove fake influencer

# Scrape Test

In [16]:
#base_url = "https://www.instagram.com/graphql/query/?query_id=17864450716183058&variables=%7B%22shortcode%22%3A%22{}%22%2C%22first%22%3A100%7D"

In [48]:
url_likes = "https://www.instagram.com/graphql/query/?query_id=17864450716183058&variables=%7B%22shortcode%22%3A%22{}%22%2C%22first%22%3A{}%7D"

In [6]:
driver = webdriver.Firefox()

In [7]:
selenium_instagram_login(driver, 'instagram_credentials.json')

In [34]:
driver.get(url_likes.format(influencers_top_post[top[0]]['post'], str(influencers_top_post[top[0]]['max_likes'])))

In [35]:
driver.find_element_by_id('tab-1').click()

In [36]:
data = driver.find_element_by_css_selector('pre.data')

In [37]:
my_json = json.loads(data.text)

In [40]:
likes = my_json['data']['shortcode_media']['edge_liked_by']['edges']

In [43]:
collection.insert_one({'id':top[0],'likes': likes})

In [ ]:
# Removed this record from mongo db

# SCRAPE THE REST!

In [49]:
for i in range(len(top)):
    driver.get(url_likes.format(influencers_top_post[top[i]]['post'], str(influencers_top_post[top[i]]['max_likes'])))
    time.sleep(np.random.uniform(2,4))
    
    driver.find_element_by_id('tab-1').click()
    data = driver.find_element_by_css_selector('pre.data')
    my_json = json.loads(data.text)
    likes = my_json['data']['shortcode_media']['edge_liked_by']['edges']
    collection.insert_one({'id':top[i],'likes': likes})
    
    time.sleep(np.random.uniform(5,8))

# Saving the collected data

In [67]:
cursor = collection.find({})
for x in cursor:
    user_id = x['id']
    likes = []
    for liker in x['likes']:
        likes.append(liker['node']['id'])
    influencers_top_post[user_id]['likes'] = likes

In [69]:
write_json(influencers_top_post, 'data/influencers_top_posts_likes.json')

# Checking the data

In [72]:
for k,v in influencers_top_post.iteritems():
    print k, v['max_likes'], len(v['likes'])

3935144838 1597 1596
47296290 3158 3158
5090616 3159 3159
4052294333 469 469
847685806 2693 2693
1474420 966 966
1977468026 3647 3647
11295358 5330 5330
612796387 1248 1248
481991581 2298 2297
1059091782 4419 4419
19263527 679 679
3125942337 898 898
1257079395 3928 3928
1495667341 4619 4619
2218921 808 808
622184175 318 318
458602313 2142 2142
198585309 3433 3433
4470742668 114 114
456656920 1827 1827
555987737 677 677
5695204531 134 134
53440747 1434 1434
53828060 1207 1207
320187471 222 222
1770521683 433 433
1279708751 207 207
